In [ ]:
# This is based on the pilot on the UC Berkeley campus in Fall 2017
# To obtain the data that this is based on, for research and reproducibility,
# request data from Sep 2017 to Dec 2017 using
# https://github.com/e-mission/e-mission-server/wiki/Requesting-data-as-a-collaborator

# Setup

In [ ]:
import pandas as pd
import arrow
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import sys
%matplotlib inline

In [ ]:
import emission.storage.timeseries.aggregate_timeseries as estag
import emission.storage.timeseries.timequery as estt

In [ ]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

In [ ]:
agts = estag.AggregateTimeSeries()
sep_dec_tq_data_ts = estt.TimeQuery("data.ts", arrow.get("2016-09-01").timestamp, arrow.get("2016-12-31").timestamp)
print arrow.get(sep_dec_tq_data_ts.startTs), "->", arrow.get(sep_dec_tq_data_ts.endTs), "=", sep_dec_tq_data_ts.get_query()

### Reading the server API calls

In [ ]:
server_api_calls_df = agts.get_data_df("stats/server_api_time", time_query=sep_dec_tq_data_ts)

In [ ]:
len(server_api_calls_df)

In [ ]:
server_api_calls_df.head()

In [ ]:
server_api_calls_df['fmt_utc_time'] = server_api_calls_df.ts.apply(lambda t: arrow.get(t).format())

In [ ]:
server_api_calls_df.head()

### Splitting into timeline and non-timeline

Several of the calls are of the form `POST_/timeline/getTrips/2016-12-28`. These are all just calls to read the timeline, but because the id is part of the URL, they are treated as unique. Let's split them from the other callbacks so that we can see which calls actually happened.

In [ ]:
unique_api_calls = pd.Series(server_api_calls_df.name.unique())

In [ ]:
unique_api_calls.head()

In [ ]:
timeline_call_mask = np.empty(len(unique_api_calls))
timeline_call_mask.fill(False)
# Needed because if it is not there, fill(False) fills with zeros
# I have no idea why. Double negation fixes it.
timeline_call_mask = np.logical_not(np.logical_not(timeline_call_mask))
print timeline_call_mask[0:5]
len(timeline_call_mask), len(unique_api_calls)

In [ ]:
for i, call in enumerate(unique_api_calls):
    if call.startswith("POST_/timeline"):
        timeline_call_mask[i] = True

In [ ]:
np.count_nonzero(timeline_call_mask), np.count_nonzero(np.logical_not(timeline_call_mask))

In [ ]:
timeline_call_mask[0:5], np.logical_not(timeline_call_mask)[0:5]

In [ ]:
unique_api_calls[timeline_call_mask].head()

In [ ]:
timeline_calls_list = list(unique_api_calls[timeline_call_mask])

In [ ]:
non_timeline_calls_list = list(unique_api_calls[np.logical_not(timeline_call_mask)])

In [ ]:
timeline_calls_list[0:5], non_timeline_calls_list[0:5]

In [ ]:
non_timeline_calls_list

### Some sanity checking

We are only supposed to have data from this Sept to this Dec, at which time the apps in the stores were the latest version, but we still see calls to `/tripManager/getUnclassifiedSections` and `/compare`, which were calls from the old version of the apps. Amazingly enough, we even see a `_/movesCallback`, which is generated while signing in to the old app. Who was installing the old app after we switched to the new one?

In [ ]:
server_api_calls_df[server_api_calls_df.name == "POST_/compare"]

In [ ]:
server_api_calls_df[server_api_calls_df.name == "POST_/movesCallback"]

In [ ]:
server_api_calls_df[server_api_calls_df.name == "POST_/profile/settings"]

In [ ]:
server_api_calls_df[server_api_calls_df.name == "POST_/profile/consent"]

It looks like somebody installed the old app around the 26th of Oct, and some people were using previously installed versions of the app through Oct. I wonder how they got access to it. But it is also pretty clear that I can ignore these for the purposes of this analysis.

### Reading client stats

In [ ]:
client_nav_events_df = agts.get_data_df("stats/client_nav_event", time_query=sep_dec_tq_data_ts)
# client_times_df = agts.get_data_df("stats/client_time", time_query=sep_dec_tq_data_ts)

In [ ]:
client_times_df = agts.get_data_df("stats/client_time", time_query=sep_dec_tq_data_ts)

# Number of users

### Number of users who signed up in this time frame

We first get the number of calls to `/profile/create`. This is the number of sign-ups

In [ ]:
profile_create_api_calls_df = server_api_calls_df[server_api_calls_df.name == 'POST_/profile/create']

In [ ]:
profile_create_api_calls_df.head()

In [ ]:
n_profile_create_api_calls = len(profile_create_api_calls_df)
n_profile_create_api_calls

The profile create call is not associated with a user_id, so we can't see how many unique calls there were. But it is a reasonable proxy.

### Number of unique userids making calls in this timeframe

In [ ]:
unique_userid_calls = server_api_calls_df.user_id.unique()

In [ ]:
unique_userid_calls[0:5], unique_userid_calls[-5:-1]

In [ ]:
unique_userid_calls

In [ ]:
n_unique_userid_calls = len(unique_userid_calls)
n_unique_userid_calls

This includes all users who were active during that time, not just users who signed up during that time 

### Number of unique userids making calls to `/usercache/put` in this timeframe

Since it looks like there are still some calls from old clients, let's see if we can look at unique active users from the new client. We pick the `/usercache/put` call as a proxy because if there were no calls to it, we got no data.

In [ ]:
usercache_put_api_calls_df = server_api_calls_df[server_api_calls_df.name == 'POST_/usercache/put']

In [ ]:
usercache_put_api_calls_df.head()

In [ ]:
n_unique_userid_usercache_put_calls = len(usercache_put_api_calls_df.user_id.unique())
n_unique_userid_usercache_put_calls

### Number of people who signed up for the game

In [ ]:
habitica_register_api_calls_df = server_api_calls_df[server_api_calls_df.name == 'POST_/habiticaRegister']

In [ ]:
n_unique_userid_habitica_register_calls = len(habitica_register_api_calls_df.user_id.unique())
n_unique_userid_habitica_register_calls

### Figure 1: Installation numbers according to various metrics

In [ ]:
result_dict = {'Number of sign-ups': n_profile_create_api_calls,
               'Number of calls from unique users': n_unique_userid_calls,
                                        'Number of unique users with new clients': n_unique_userid_usercache_put_calls,
                                        'Number of unique sign-ups for the game': n_unique_userid_habitica_register_calls}

In [ ]:
installation_numbers_df = pd.DataFrame(columns=["users"])
for k, v in result_dict.items():
    print "Adding %s -> %s" % (k, v)
    installation_numbers_df.loc[k] = v

In [ ]:
sorted_indf = installation_numbers_df.sort_values(by='users', ascending=False)
sorted_indf

In [ ]:
sorted_indf.to_latex()

In [ ]:
sorted_indf.plot(kind='bar', figsize=(18, 6), rot=0, fontsize=10)
# plt.tick_params(axis='x', labelsize=10)

### Figure 2: sign-ups versus time

Naive graph of response time versus ts

In [ ]:
profile_create_api_calls_df.plot(kind="bar", x="ts", y="reading", figsize=(16,6))

Let's try to switch to a timestamp object instead

In [ ]:
profile_create_api_calls_df['pd_ts'] = profile_create_api_calls_df.ts.apply(lambda t: pd.Timestamp(t, unit='s'))
profile_create_api_calls_df['count_col'] = np.ones(len(profile_create_api_calls_df))

In [ ]:
profile_create_api_calls_df.head()

Let's see what the graph looks like now

In [ ]:
profile_create_api_calls_df.plot(kind="bar", x="pd_ts", y="reading", figsize=(16,6))

In [ ]:
profile_create_api_calls_df.plot(kind="bar", x="pd_ts", y="count_col", figsize=(16,6))

In [ ]:
# Let's group by day to have it make sense
# Similar to https://stackoverflow.com/questions/15297053/how-can-i-divide-single-values-of-a-dataframe-by-monthly-averages
daily_profile_create_api = pd.Series()
grouper = pd.Grouper(key='pd_ts', freq='D')
grouped_result = profile_create_api_calls_df.groupby(grouper)
for g in grouped_result:
    print "%s -> %s" % (g[0], g[1].count_col.count())
    daily_profile_create_api.loc[g[0]] = g[1].count_col.count()

In [ ]:
ax = daily_profile_create_api.plot(kind='bar', figsize=(16,6), fontsize=12)
ax.set_xlabel("Date", fontsize=15)
ax.xaxis.set_major_formatter(plt.FixedFormatter(daily_profile_create_api.index.to_series().dt.strftime("%d %b %Y")))
ax.set_ylabel("number of sign-up calls per day", fontsize=15)

# Length of install 

Rough algorithm:
for each user, find the first and last call to `/usercache/put`. difference between them is the length of install

### Exploratory analysis for a single user

In [ ]:
from uuid import UUID
# def get_install_length(user_id):
user_usercache_put_calls = server_api_calls_df[(server_api_calls_df.name == 'POST_/usercache/put') & (server_api_calls_df.user_id == UUID('30ede46c-3b80-4ebb-aa3a-38e78e3c08d7'))]
user_usercache_put_calls.head()

In [ ]:
sorted_user_usercache_put_calls = user_usercache_put_calls.sort_values(by="ts", ascending=True)
sorted_user_usercache_put_calls.head()

In [ ]:
sorted_user_usercache_put_calls.tail()

In [ ]:
first_ts = sorted_user_usercache_put_calls.ts.iloc[0]
last_ts = sorted_user_usercache_put_calls.ts.iloc[-1]
delta = last_ts - first_ts
duration = pd.Timedelta(delta, unit='s')
delta, duration, duration.days

But note that the last entries are around the end of our analysis time period

In [ ]:
last_dt = pd.to_datetime(last_ts, unit='s')
last_dt

In [ ]:
last_dt.month, last_dt.day

### Final function for a single user

In [ ]:
def get_install_length(user_id):
    user_usercache_put_calls = server_api_calls_df[(server_api_calls_df.name == 'POST_/usercache/put') & (server_api_calls_df.user_id == user_id)]
    if len(user_usercache_put_calls) == 0:
        return 0
    sorted_user_usercache_put_calls = user_usercache_put_calls.sort_values(by="ts", ascending=True)
    first_ts = sorted_user_usercache_put_calls.ts.iloc[0]
    last_ts = sorted_user_usercache_put_calls.ts.iloc[-1]
    print "considering duration %s -> %s" % (arrow.get(first_ts), arrow.get(last_ts))
    last_dt = pd.to_datetime(last_ts, unit='s')
    if last_dt.month == 12 and last_dt.day > 29:
        return sys.maxint
    else:
        delta = last_ts - first_ts
        duration = pd.Timedelta(delta, unit='s')
        return duration.days

Now, we get make a new dataframe for the results

In [ ]:
install_duration = pd.DataFrame(columns=["start_ts", "end_ts", "duration"])
for u in usercache_put_api_calls_df.user_id.unique():
    start_end_duration = get_install_length(u)
    print "%s -> %s" % (u, start_end_duration)
    install_duration.loc[u] = start_end_duration

In [ ]:
install_duration.head()

In [ ]:
install_duration[install_duration.duration == sys.maxint].count()

In [ ]:
install_duration[install_duration.duration == 0].count()

In [ ]:
display_install_duration = install_duration[install_duration.duration != sys.maxint]

In [ ]:
ax = display_install_duration.duration.plot(kind='hist', label='Number of users', fill=False, figsize=(16,6), bins=100, fontsize=12)
ax2 = display_install_duration.duration.plot(kind='hist', label="Cumulative users", ax=ax, cumulative=-1, normed=True, histtype='step', secondary_y=True, bins=100)
ax.set_xlabel("Install duration", fontsize=15)
ax.set_ylabel("Number of users", fontsize=15)
# ax.axhline(y=23, color='black')
ax.text(40, 15, "23 users did not uninstall, install duration unknown")
ax.annotate("Spurious installs?", xy=(0,16), xytext=(5,15), arrowprops=dict(facecolor='black', shrink=0.1))
ax2.set_ylabel("Cumulative percentile of users", fontsize=15, color='g')
ax2.axhline(y=0.85, label="0.5", color='m')
ax2.text(90, 0.85, "85%", color='m')
ax2.axhline(y=0.5, label="0.5", color='r')
ax2.text(90, 0.5, "50%", color='r')
ax2.axhline(y=0.3, label="0.3", color='b')
ax2.text(90, 0.3, "30%", color='b')
ax.axvline(x=3, color='m')
ax.text(3, 5, "3 days", color='m')
ax.axvline(x=20, color='r')
ax.text(20, 5, "20 days", color='r')
ax.axvline(x=38, color='b')
ax.text(38, 5, "38 days", color='b')
ax.legend()

# User interaction

### Number of times app was launched

#### One way: look at calls to the dashboard.

this can either be `/result/metrics/timestamp` or `/result/metrics/local_date`

Let's figure out which one is the default

In [ ]:
metrics_timestamp_calls = server_api_calls_df[server_api_calls_df.name == 'POST_/result/metrics/timestamp']
metrics_timestamp_calls.count()

In [ ]:
server_api_calls_df[server_api_calls_df.name == 'POST_/result/metrics/local_date'].count()

Timestamp is clearly the default and there were 6207 opens

#### Second way: look at `app_launched` stats

In [ ]:
app_launched_events = client_nav_events_df[client_nav_events_df.name == "app_launched"]
app_launched_events.count()

In [ ]:
metrics_timestamp_calls['pd_ts'] = metrics_timestamp_calls.ts.apply(lambda t: pd.Timestamp(t, unit='s'))
app_launched_events['pd_ts'] = app_launched_events.ts.apply(lambda t: pd.Timestamp(t, unit='s'))

#### Briefly try to explore the discrepancy

In [ ]:
metrics_timestamp_calls.head()

In [ ]:
app_launched_events.head()

In [ ]:
metrics_timestamp_calls[metrics_timestamp_calls.user_id == UUID('68d16b1a-1521-4eaf-922a-20ba037cdc79')].head()

In [ ]:
app_launched_events[app_launched_events.user_id == UUID('68d16b1a-1521-4eaf-922a-20ba037cdc79')].head()

Looks like the server calls are more accurate, let's use them for the next analysis

#### Figure 1: plot both on the same graph, let's group by day again

In [ ]:
# Let's group by day to have it make sense
# Similar to https://stackoverflow.com/questions/15297053/how-can-i-divide-single-values-of-a-dataframe-by-monthly-averages
daily_launches = pd.Series()
grouper = pd.Grouper(key='pd_ts', freq='w')
for g in metrics_timestamp_calls.groupby(grouper):
    print "%s -> %s" % (g[0], g[1].ts.count())
    daily_launches.loc[g[0]] = g[1].ts.count()

In [ ]:
ax = daily_launches.plot(kind='bar', figsize=(16,6), fontsize=12)
ax.set_xlabel("Date", fontsize=15)
ax.xaxis.set_major_formatter(plt.FixedFormatter(daily_launches.index.to_series().dt.strftime("%d %b %Y")))
ax.set_ylabel("number of app launch calls per week", fontsize=15)

In [ ]:
user_calls = pd.Series()
for g in metrics_timestamp_calls.groupby(by="user_id"):
    print "%s -> %s" % (g[0], g[1].ts.count())
    user_calls.loc[g[0]] = g[1].ts.count()    

In [ ]:
user_calls.head()

In [ ]:
ax = user_calls[user_calls.index != ''].plot(kind='hist', label='Number of launches/user', fill=False, figsize=(16,6), bins=600, fontsize=12)
ax2 = user_calls[user_calls.index != ''].plot(kind='hist', label="Cumulative fraction", ax=ax, cumulative=-1, normed=True, histtype='step', secondary_y=True, bins=600)
ax.set_xlabel("Number of app launches", fontsize=15)
ax.set_ylabel("Number of users", fontsize=15)
ax2.set_ylabel("Cumulative percentile of users", fontsize=15, color='g')
ax2.axhline(y=0.8, label="0.5", color='r')
ax2.text(550, 0.8, "80%", color='r')
ax2.axhline(y=0.5, label="0.3", color='b')
ax2.text(550, 0.5, "50%", color='b')
ax2.axhline(y=0.1, label="0.1", color='m')
ax2.text(550, 0.1, "10%", color='m')
ax.axvline(x=5, color='r')
ax.text(5, 11, "5 launches", color='r')
ax.axvline(x=20, color='b')
ax.text(20, 10, "20 launches", color='b')
ax.axvline(x=150, color='m')
ax.text(150, 9, "150 launches", color='m')
ax.legend()

### Number of times user navigated to another window

In [ ]:
state_changed_events = client_times_df[client_times_df.name == "state_changed"]
print len(state_changed_events)
state_changed_events.head()

In [ ]:
# Double negative to make it boolean instead of 0/1
non_splash_mask = np.logical_not(np.logical_not(np.zeros(len(state_changed_events))))
non_splash_mask[0:5]

In [ ]:
state_changed_events.reading.head()

In [ ]:
for i, e in enumerate(state_changed_events.reading):
    if "splash" in e:
        non_splash_mask[i] = False
    else:
        non_splash_mask[i] = True

In [ ]:
non_splash_mask[0:5]

In [ ]:
state_changed_events.index[non_splash_mask]

In [ ]:
non_splash_state_change_events = state_changed_events[state_changed_events.index.isin(state_changed_events.index[non_splash_mask])]
non_splash_state_change_events.head()

In [ ]:
non_splash_state_change_events['pd_ts'] = non_splash_state_change_events.ts.apply(lambda t: pd.Timestamp(t, unit='s'))

#### Figure 2: Plot against time

In [ ]:
# Let's group by day to have it make sense
# Similar to https://stackoverflow.com/questions/15297053/how-can-i-divide-single-values-of-a-dataframe-by-monthly-averages
daily_screen_switches = pd.Series()
grouper = pd.Grouper(key='pd_ts', freq='D')
for g in non_splash_state_change_events.groupby(grouper):
    print "%s -> %s" % (g[0], g[1].ts.count())
    daily_screen_switches.loc[g[0]] = g[1].ts.count()

In [ ]:
ax = daily_screen_switches.plot(kind='bar', figsize=(16,6), fontsize=12)
ax.set_xlabel("Date", fontsize=15)
ax.xaxis.set_major_formatter(plt.FixedFormatter(daily_screen_switches.index.to_series().dt.strftime("%d %b %Y")))
ax.set_ylabel("number of screen switches per day", fontsize=15)